<a href="https://colab.research.google.com/github/avaneeshp23/GenAIhackathon/blob/hackathon/classification_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install groq

In [17]:
import os
os.environ['GROQ_API_KEY'] = "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"

In [18]:
from groq import Groq
import pandas as pd
import json

In [19]:
client = Groq(api_key=os.environ['GROQ_API_KEY'])

In [20]:
!git clone https://github.com/avaneeshp23/GenAIhackathon.git

fatal: destination path 'GenAIhackathon' already exists and is not an empty directory.


In [33]:
def classify_ticket(text):
    prompt = f"""
    Classify the following IT ticket text according to these instructions:

    **Department:** Choose from: Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry.
    **Priority:** Determine the urgency level: low, medium, or high.
    **Language:** Identify the language code (e.g., en, de, es).  Refer to language codes for guidance.
    **Type:** Categorize the nature of the request: Incident, Request, Problem, or Change.

    Text: {text}

    Provide the output in JSON format:
    {{
    "Department": "...",
    "Priority": "...",
    "Language": "...",
    "Type": "..."
    }}
    """
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
    )
    print("Response", response)
    classification_result = response.choices[0].message.content.strip()
    return classification_result

In [34]:
import time
import logging
import pandas as pd

def classify_ticket_safe(text, max_retries=5, base_wait=100):
    """
    Calls classify_ticket with automatic retries on RateLimitError.

    Args:
        text (str): The text to classify.
        max_retries (int): Maximum number of retries.
        base_wait (int): Initial wait time in seconds.

    Returns:
        str: The classification result.
    """
    for attempt in range(max_retries):
        try:
            return classify_ticket(text)  # Call your function
        except RateLimitError as e:
            wait_time = base_wait * (2 ** attempt)  # Exponential backoff
            logging.warning(f"Rate limit hit (Attempt {attempt+1}/{max_retries}). Waiting {wait_time} seconds...")
            time.sleep(wait_time)

    logging.error("Max retries reached. Returning None.")
    return None  # Return None if all retries fail

In [35]:
train_df = pd.read_csv('/content/GenAIhackathon/data/train.csv')

In [36]:
train_df['text'] = train_df['ticket_subject'] + ' ' + train_df['ticket_body']
train_df = train_df[['text', 'department', 'type', 'priority', 'language']]

In [37]:
train_df['classification'] = train_df['text'].apply(classify_ticket)

Response ChatCompletion(id='chatcmpl-36dc97a2-f4bb-4388-804a-5867a5f006b8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is the classification of the IT ticket text in JSON format:\n\n{\n"Department": "Billing and Payments",\n"Priority": "High",\n"Language": "es",\n"Type": "Request"\n}\n\nExplanation:\n\n* Department: Billing and Payments, as the ticket is about an incorrect billing issue with Google Workspace.\n* Priority: High, as the customer is expecting a prompt response and resolution to the issue.\n* Language: es, as the ticket is written in Spanish.\n* Type: Request, as the customer is making a request to have their invoice corrected.', role='assistant', function_call=None, reasoning=None, tool_calls=None))], created=1739127191, model='llama3-8b-8192', object='chat.completion', system_fingerprint='fp_6a6771ae9c', usage=CompletionUsage(completion_tokens=114, prompt_tokens=252, total_tokens=366, completion_time=0.095, p

In [38]:
train_df['classification'][0]

'Here is the classification of the IT ticket text in JSON format:\n\n{\n"Department": "Billing and Payments",\n"Priority": "High",\n"Language": "es",\n"Type": "Request"\n}\n\nExplanation:\n\n* Department: Billing and Payments, as the ticket is about an incorrect billing issue with Google Workspace.\n* Priority: High, as the customer is expecting a prompt response and resolution to the issue.\n* Language: es, as the ticket is written in Spanish.\n* Type: Request, as the customer is making a request to have their invoice corrected.'

In [39]:
test_df = pd.read_csv('/content/GenAIhackathon/data/test.csv')
test_df.head()

,ticket_ID,ticket_subject,ticket_body
0,1,Urgente: problema con la funzione di condivisi...,"Gentile assistenza clienti,\n\nSto riscontrand..."
1,2,Abrechnungsdiskrepanzen in Google Workspace,"Liebes IT-Support-Team,\n\nMit diesem Schreibe..."
2,3,Problèmes avec la fonction sans fil de l'Epson...,"Bonjour à l'équipe d'assistance,\n\nJ'ai un pr..."
3,4,NaN,"Gentile assistenza clienti, \n\nSpero che ques..."
4,5,Assistenza necessaria per gli arresti anomali ...,Gentile assistenza clienti di Tech Online Stor...


In [56]:
test_df['text'] = test_df['ticket_subject'].fillna('') + ' ' + test_df['ticket_body'].fillna('')
test_df.shape

(500, 4)

In [57]:
# small_df = test_df[:5]
# small_df.head()
# small_df['classification'] = small_df['text'].apply(classify_ticket)

Response ChatCompletion(id='chatcmpl-3d5a88e4-579b-4435-ba9b-180f26c05a4f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here is the classification of the IT ticket text in JSON format:\n\n{\n  "Department": "IT Support",\n  "Priority": "High",\n  "Language": "it",\n  "Type": "Incident"\n}\n\nHere\'s a breakdown of the classification:\n\n* Department: The text describes a technical issue with the Zoom screen sharing function, which falls under the purview of IT Support.\n* Priority: The text explicitly states that the incident is of high priority, as it\'s interfering with critical business operations and requires immediate attention.\n* Language: The text is written in Italian, which corresponds to the language code "it".\n* Type: The text describes a sudden failure or interruption of a service (screen sharing in Zoom), which is classified as an "Incident".', role='assistant', function_call=None, reasoning=None, tool_calls=None))

<ipython-input-57-7ee87ae013db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_df['classification'] = small_df['text'].apply(classify_ticket)


In [42]:
# import re
# def extract_json_from_classification(classification_text):
#     try:
#         # Extract JSON part from the classification text
#         json_text = re.search(r'\{.*\}', classification_text, re.DOTALL).group()
#         data = json.loads(json_text)
#         return data
#     except (json.JSONDecodeError, AttributeError) as e:
#         logging.error(f"Error extracting JSON: {e}")
#         return None

# def extract_values_with_id(row):
#     data = extract_json_from_classification(row['classification'])
#     if data:
#         return pd.Series({
#             'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
#             'department': data.get('Department', None),
#             'type': data.get('Type', None),
#             'priority': data.get('Priority', None),
#             'language': data.get('Language', None)
#         })
#     else:
#         return pd.Series({
#             'ticket_ID': row['ticket_ID'],  # Include ticketID for mapping
#             'department': None,
#             'type': None,
#             'priority': None,
#             'language': None
#         })

In [ ]:
test_df['classification'] = test_df['text'].apply(classify_ticket)

In [68]:
import re
import json
import logging
import pandas as pd

logging.basicConfig(level=logging.ERROR)

def clean_json_text(json_text):
    """
    Cleans JSON-like text by fixing common formatting issues.

    Fixes:
    - Removes inline comments (e.g., `// ...`).
    - Removes inline annotations in parentheses (e.g., `"Language": "it" (Italian)` → `"Language": "it"`).
    - Replaces single quotes (`'`) with double quotes (`"`).
    - Removes trailing commas (e.g., `,}` → `}`).

    Args:
        json_text (str): Raw extracted JSON-like text.

    Returns:
        str: Cleaned JSON text.
    """
    try:
        json_text = re.sub(r'//.*', '', json_text)

        json_text = re.sub(r'("\s*:\s*"[^"]+)"\s*\([^)]*\)', r'\1"', json_text)

        json_text = re.sub(r"(?<!\\)'", '"', json_text)

        json_text = re.sub(r',\s*([\]}])', r'\1', json_text)

        return json_text
    except Exception as e:
        logging.error(f"Error cleaning JSON text: {e}")
        return None

def extract_json_from_classification(classification_text):
    """
    Extracts JSON content from a classification response and sanitizes it.

    Args:
        classification_text (str): The text containing JSON.

    Returns:
        dict or None: Extracted JSON data as a dictionary, or None if extraction fails.
    """
    try:
        # Extract JSON using regex
        json_match = re.search(r'\{.*\}', classification_text, re.DOTALL)
        if json_match:
            json_text = json_match.group()

            # Clean the JSON before parsing
            json_text = clean_json_text(json_text)

            # Parse JSON
            return json.loads(json_text)
        else:
            logging.error("No valid JSON found in the response.")
            return None
    except json.JSONDecodeError as e:
        logging.error(f"Error parsing JSON: {e}\nJSON Text: {json_text}")
        return None

def extract_values_with_id(row):
    """
    Extracts structured data from the model response and returns a Series.

    Args:
        row (pd.Series): A row of the DataFrame.

    Returns:
        pd.Series: Extracted values mapped to columns.
    """
    data = extract_json_from_classification(row["classification"])
    return pd.Series({
        "ticket_ID": row["ticket_ID"],
        "department": data.get("Department", None) if data else None,
        "type": data.get("Type", None) if data else None,
        "priority": data.get("Priority", None) if data else None,
        "language": data.get("Language", None) if data else None
    })

In [69]:
# Apply function to extract values and create multiple columns
predictions_df = test_df.apply(extract_values_with_id, axis=1)
predictions_df.head()

,ticket_ID,department,type,priority,language
0,1,IT Support,Incident,High,it
1,2,Billing and Payments,Request,low,de
2,3,IT Support,Incident,High,fr
3,4,Technical Support,Request,medium,it
4,5,IT Support,Incident,Medium,it


In [70]:
submission_path = '/content/GenAIhackathon/data/submission.csv'
predictions_df.to_csv(submission_path, index= False)

In [71]:
submission_df = pd.read_csv(submission_path)
submission_df.head()

,ticket_ID,department,type,priority,language
0,1,IT Support,Incident,High,it
1,2,Billing and Payments,Request,low,de
2,3,IT Support,Incident,High,fr
3,4,Technical Support,Request,medium,it
4,5,IT Support,Incident,Medium,it
